In [3]:
import prosperity3bt
import pandas as pd
import itertools
import numpy as np
from tqdm import tqdm

from prosperity3bt.runner import run_backtest
from prosperity3bt.models import TradeMatchingMode
from prosperity3bt.file_reader import FileSystemReader
from pathlib import Path

from importlib import reload

In [8]:
def day_profit(result):
    last_timestamp = result.activity_logs[-1].timestamp

    profit = 0
    for row in reversed(result.activity_logs):
        if row.timestamp != last_timestamp:
            break

        profit += row.columns[-1]

    return profit

def backtest_days(trader, days=[-2, -1, 0], data_path='.', round_num=1):
    profits = []
    for day in days:
        backtest_results = prosperity3bt.runner.run_backtest(trader, file_reader=FileSystemReader(Path(data_path)), round_num=round_num, day_num=day, print_output=False, trade_matching_mode=TradeMatchingMode.all, no_names=True, show_progress_bar=False)
        profit = day_profit(backtest_results)
        profits.append(profit)
    return profits

def generate_param_combinations(param_grid):
    param_names = param_grid.keys()
    param_values = param_grid.values()
    combinations = list(itertools.product(*param_values))
    return [dict(zip(param_names, combination)) for combination in combinations]

In [9]:
import cookin_ink

reload(cookin_ink)


DEFAULT_PARAMS = {
    'ink_lookback_window': 10,            # Number of periods for momentum lookback
    'ink_momentum_threshold': 0.008,      # Change in return required to trigger a trade
    'ink_past_lag': 1,                     # Number of periods to look back for past return
    'ink_clear_threshold': 0.0001,         # Threshold below which we clear the position,
}

param_grid = {
    'ink_lookback_window': [5, 8, 10, 12, 15, 20, 25, 30],
    'ink_momentum_threshold': [0.005, 0.007, 0.008, 0.009, 0.01, 0.012, 0.015],
    'ink_past_lag': [1, 2, 3, 4, 5],
    'ink_clear_threshold': [0.0001],
}

param_profits = []

for params in tqdm(generate_param_combinations(param_grid)):
    trader = cookin_ink.Trader(params)
    days = [1, 2, 3]
    profits = backtest_days(trader, days=days, data_path='.', round_num=4)
    for d in days:
        params[f'profit_day_{d}'] = profits[days.index(d)]
    param_profits.append(params)

  0%|          | 0/280 [00:00<?, ?it/s]

100%|██████████| 280/280 [25:53<00:00,  5.55s/it]


In [ ]:
# TOOD find sharpe for best stable pnl

In [11]:
df = pd.DataFrame(param_profits)
df['tot_profit'] = df[[f'profit_day_{d}' for d in days]].sum(axis=1)
df.sort_values(by='tot_profit', ascending=False, inplace=True)
df

,ink_lookback_window,ink_momentum_threshold,ink_past_lag,ink_clear_threshold,profit_day_1,profit_day_2,profit_day_3,tot_profit
175,20,0.005,1,0.0001,2465.0,6274.0,2807.0,11546.0
102,10,0.015,3,0.0001,3129.0,5701.5,-361.0,8469.5
97,10,0.012,3,0.0001,3129.0,5806.5,-533.5,8402.0
101,10,0.015,2,0.0001,3129.0,4932.0,-528.5,7532.5
96,10,0.012,2,0.0001,3129.0,5024.0,-768.0,7385.0
...,...,...,...,...,...,...,...,...
3,5,0.005,4,0.0001,-8554.0,-14305.0,-6667.0,-29526.0
109,12,0.005,5,0.0001,-12562.0,-11774.0,-6486.5,-30822.5
9,5,0.007,5,0.0001,-9793.0,-10469.0,-10726.0,-30988.0
4,5,0.005,5,0.0001,-8845.0,-11950.5,-10355.5,-31151.0


In [7]:
df.to_csv('backtests/ink_trader_momentum_day1.csv')